## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([1, 1, 1,  ..., 1, 0, 0], dtype=torch.uint8)

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [0]:
def get_parallel_db(db, remove_index):
  
  return torch.cat((db[0:remove_index],
                    db[remove_index+1:]))

In [0]:
def get_parallel_dbs(db):
  
  parallel_dbs = list()
  
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
    
  return parallel_dbs

In [0]:
def create_db_and_parallels(num_entries):
  
  db = torch.rand(num_entries) > 0.5
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs 

In [5]:
db, pdbs = create_db_and_parallels(6)
db

tensor([1, 0, 1, 0, 1, 1], dtype=torch.uint8)

In [6]:
pdbs

[tensor([0, 1, 0, 1, 1], dtype=torch.uint8),
 tensor([1, 1, 0, 1, 1], dtype=torch.uint8),
 tensor([1, 0, 0, 1, 1], dtype=torch.uint8),
 tensor([1, 0, 1, 1, 1], dtype=torch.uint8),
 tensor([1, 0, 1, 0, 1], dtype=torch.uint8),
 tensor([1, 0, 1, 0, 1], dtype=torch.uint8)]

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [0]:
db, pdbs = create_db_and_parallels(5000)

In [0]:
def query(db):
    return db.sum()

In [0]:
full_db_result = query(db)

In [0]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [11]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [0]:
def sensitivity(query, n_entries=1000):

    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)
    
        db_distance = torch.abs(pdb_result - full_db_result)
    
        if(db_distance > max_distance):
            max_distance = db_distance
    
    return max_distance

In [0]:
def query(db):
    return db.float().mean()

In [14]:
sensitivity(query)

tensor(0.0005)

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [0]:
def query(db, threshold=5):
    return (db.sum()>threshold).float()

In [16]:
for i in range(10):
    sens_f =  sensitivity(query, n_entries=10)
    print(sens_f)

0
0
0
0
0
0
0
0
tensor(1.)
0


# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [0]:
db, _ = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db, remove_index=10)

In [19]:
# differencing attack using sum query

sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [20]:
# differencing attack using sum query

(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0049)

In [21]:
# differencing attack using threshold

(sum(db).float() > 49) - (sum(pdb).float() >49)

tensor(0, dtype=torch.uint8)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [0]:
db, pdbs = create_db_and_parallels(100)

In [23]:
db

tensor([1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0], dtype=torch.uint8)

In [0]:
def query(db):
    
    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    
    augmented_database = db.float()*first_coin_flip + (1-first_coin_flip)*second_coin_flip
    
    db_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
    return db_result, true_result

In [25]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5000)
Without noise:tensor(0.5000)


In [26]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.6200)
Without noise:tensor(0.4900)


In [27]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.4980)
Without noise:tensor(0.4890)


# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [0]:
def query(db, noise=0.2):
    
    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    
    augmented_database = db.float()*first_coin_flip + (1-first_coin_flip)*second_coin_flip
    
    sk_result = augmented_database.float().mean() 
    
    private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)
        
    return private_result, true_result

In [32]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db, noise=0.1)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5178)
Without noise:tensor(0.5190)


In [33]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.2)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.4625)
Without noise:tensor(0.4800)


In [34]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.4)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5333)
Without noise:tensor(0.5100)
